1. Import thư viện:

In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

2. Đọc và tiền xử lý dữ liệu:

In [2]:
# Đọc dữ liệu
df = pd.read_csv('uber.csv')

def haversine_distance(lat1, lon1, lat2, lon2):
    """Tính khoảng cách giữa hai điểm trên trái đất"""
    R = 3959.87433  # Bán kính trái đất tính bằng dặm
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    mi = R * c
    return mi

# Tiền xử lý dữ liệu
print("Bắt đầu tiền xử lý dữ liệu...")
print(f"Kích thước ban đầu: {df.shape}")

# Chuyển đổi pickup_datetime thành datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Trích xuất các đặc trưng thời gian
df['hour'] = df['pickup_datetime'].dt.hour
df['day'] = df['pickup_datetime'].dt.day
df['month'] = df['pickup_datetime'].dt.month
df['year'] = df['pickup_datetime'].dt.year
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek

# Tính khoảng cách
df['distance'] = haversine_distance(
    df['pickup_latitude'], 
    df['pickup_longitude'],
    df['dropoff_latitude'],
    df['dropoff_longitude']
)

# Thêm feature mới: khoảng cách bình phương để xử lý phi tuyến
df['distance_squared'] = df['distance'] ** 2

# Thêm feature tương tác
df['distance_passenger'] = df['distance'] * df['passenger_count']

# Lọc dữ liệu theo tiêu chí
df_cleaned = df[
    (df['fare_amount'] > 0) & 
    (df['fare_amount'] <= 100) &
    (df['passenger_count'] > 0) &
    (df['passenger_count'] <= 6) &
    (df['distance'] <= 31) &
    (df['pickup_longitude'].between(-74.03, -73.77)) &
    (df['pickup_latitude'].between(40.63, 40.85)) &
    (df['dropoff_longitude'].between(-74.03, -73.77)) &
    (df['dropoff_latitude'].between(40.63, 40.85))
]

print(f"Kích thước sau khi làm sạch: {df_cleaned.shape}")

Bắt đầu tiền xử lý dữ liệu...
Kích thước ban đầu: (200000, 9)
Kích thước sau khi làm sạch: (192581, 17)


In [9]:
# Tiền xử lý dữ liệu chi tiết hơn
print("Bắt đầu tiền xử lý dữ liệu...")
print(f"Kích thước ban đầu: {df.shape}")

# 1. Kiểm tra và hiển thị thông tin về dữ liệu trước khi lọc
print("\nThống kê trước khi lọc:")
print(df[['fare_amount', 'distance', 'passenger_count']].describe())

# 2. Lọc dữ liệu với các điều kiện chặt chẽ hơn
df_cleaned = df[
    # Điều kiện về giá cước
    (df['fare_amount'] > 2.5) &                # Giá tối thiểu hợp lý
    (df['fare_amount'] <= 100) &               # Giá tối đa hợp lý
    
    # Điều kiện về số hành khách
    (df['passenger_count'] >= 1) &             # Ít nhất 1 hành khách
    (df['passenger_count'] <= 6) &             # Tối đa 6 hành khách
    
    # Điều kiện về khoảng cách
    (df['distance'] > 0) &                     # Khoảng cách phải dương
    (df['distance'] <= 31) &                   # Giới hạn khoảng cách hợp lý
    
    # Điều kiện về tốc độ trung bình (distance/time)
    (df['distance']/(df['fare_amount'] + 1e-10) <= 3) &  # Tốc độ không quá cao
    (df['distance']/(df['fare_amount'] + 1e-10) >= 0.01) &  # Tốc độ không quá thấp
    
    # Điều kiện về giá trên km
    (df['fare_amount']/df['distance'] <= 10) &  # Giá/km không quá cao
    (df['fare_amount']/df['distance'] >= 1) &   # Giá/km không quá thấp
    
    # Điều kiện về tọa độ (trong phạm vi New York)
    (df['pickup_longitude'].between(-74.03, -73.77)) &
    (df['pickup_latitude'].between(40.63, 40.85)) &
    (df['dropoff_longitude'].between(-74.03, -73.77)) &
    (df['dropoff_latitude'].between(40.63, 40.85))
]

# 3. Xử lý outliers bằng IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Áp dụng IQR cho các cột quan trọng
df_cleaned = remove_outliers_iqr(df_cleaned, 'fare_amount')
df_cleaned = remove_outliers_iqr(df_cleaned, 'distance')

# 4. Hiển thị thông tin sau khi lọc
print("\nThống kê sau khi lọc:")
print(df_cleaned[['fare_amount', 'distance', 'passenger_count']].describe())
print(f"\nKích thước sau khi làm sạch: {df_cleaned.shape}")

# 5. Tính phần trăm dữ liệu đã bị loại bỏ
percent_removed = ((df.shape[0] - df_cleaned.shape[0]) / df.shape[0]) * 100
print(f"\nPhần trăm dữ liệu đã bị loại bỏ: {percent_removed:.2f}%")

# 6. Vẽ biểu đồ phân phối trước và sau khi lọc
plt.figure(figsize=(15, 5))

# Fare Amount
plt.subplot(1, 3, 1)
plt.hist(df['fare_amount'], bins=50, alpha=0.5, label='Trước')
plt.hist(df_cleaned['fare_amount'], bins=50, alpha=0.5, label='Sau')
plt.title('Phân phối Fare Amount')
plt.legend()

# Distance
plt.subplot(1, 3, 2)
plt.hist(df['distance'], bins=50, alpha=0.5, label='Trước')
plt.hist(df_cleaned['distance'], bins=50, alpha=0.5, label='Sau')
plt.title('Phân phối Distance')
plt.legend()

# Fare/Distance Ratio
plt.subplot(1, 3, 3)
plt.hist(df['fare_amount']/df['distance'], bins=50, alpha=0.5, label='Trước')
plt.hist(df_cleaned['fare_amount']/df_cleaned['distance'], bins=50, alpha=0.5, label='Sau')
plt.title('Phân phối Fare/Distance')
plt.legend()

plt.tight_layout()
plt.show()

Bắt đầu tiền xử lý dữ liệu...
Kích thước ban đầu: (200000, 17)

Thống kê trước khi lọc:
         fare_amount       distance  passenger_count
count  200000.000000  199999.000000    200000.000000
mean       11.359955      12.962575         1.684535
std         9.901776     238.030428         1.385997
min       -52.000000       0.000000         0.000000
25%         6.000000       0.755317         1.000000
50%         8.500000       1.318296         1.000000
75%        12.500000       2.408599         2.000000
max       499.000000   10199.109218       208.000000

Thống kê sau khi lọc:
         fare_amount       distance  passenger_count
count  146120.000000  146120.000000    146120.000000
mean        8.951472       1.616591         1.682336
std         3.839089       0.911154         1.296945
min         2.900000       0.290324         1.000000
25%         6.100000       0.903651         1.000000
50%         8.100000       1.374108         1.000000
75%        11.000000       2.129250      

NameError: name 'plt' is not defined

3. Chuẩn bị dữ liệu và định nghĩa hàm đánh giá:

In [11]:
# Định nghĩa features đầu vào và giải thích
print("\nCác features được chọn:")
features = [
    'distance',          # Khoảng cách - yếu tố quan trọng nhất ảnh hưởng đến giá
    'distance_squared',  # Khoảng cách bình phương - xử lý quan hệ phi tuyến
    'distance_passenger',# Tương tác giữa khoảng cách và số hành khách
    'passenger_count',   # Số lượng hành khách
    'hour',             # Giờ trong ngày - phản ánh nhu cầu theo thời gian
    'day_of_week',      # Thứ trong tuần - phân biệt ngày làm việc/cuối tuần
    'month'             # Tháng - xu hướng theo mùa
]

for f in features:
    print(f"- {f}")

# Chuẩn bị dữ liệu
X = df_cleaned[features]
y = df_cleaned['fare_amount']  # Biến mục tiêu: giá cước taxi

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Chuẩn hóa features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def evaluate_model(y_true, y_pred, model_name):
    """Hàm đánh giá mô hình với đầy đủ 4 chỉ số"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\nKết quả đánh giá mô hình {model_name}:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print(f"MAPE: {mape:.4f}%")
    
    return mse, rmse, r2, mape


Các features được chọn:
- distance
- distance_squared
- distance_passenger
- passenger_count
- hour
- day_of_week
- month


4. Huấn luyện và đánh giá mô hình:

In [ ]:
# 1. Linear Regression
print("\nHuấn luyện Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_metrics = evaluate_model(y_test, lr_pred, "Linear Regression")

# 2. Random Forest với GridSearchCV
print("\nHuấn luyện Random Forest với GridSearchCV...")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_model = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='neg_mean_squared_error')
rf_grid.fit(X_train_scaled, y_train)

print("\nTham số tốt nhất cho Random Forest:")
print(rf_grid.best_params_)

rf_pred = rf_grid.predict(X_test_scaled)
rf_metrics = evaluate_model(y_test, rf_pred, "Random Forest (optimized)")

# So sánh feature importance của Random Forest
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_grid.best_estimator_.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTầm quan trọng của các features:")
print(feature_importance)


Huấn luyện Linear Regression...

Kết quả đánh giá mô hình Linear Regression:
MSE: 3.9600
RMSE: 1.9900
R2 Score: 0.7312
MAPE: 16.0788%

Huấn luyện Random Forest với GridSearchCV...


Random Forest với GridSearchCV

In [6]:
# Cell huấn luyện và đánh giá mô hình

# 1. Linear Regression
print("\nHuấn luyện Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_metrics = evaluate_model(y_test, lr_pred, "Linear Regression")

# 2. Random Forest với GridSearchCV được tối ưu để chạy nhanh hơn
print("\nHuấn luyện Random Forest với GridSearchCV...")
rf_params = {
    'n_estimators': [50, 100],      # Giảm số lượng cây
    'max_depth': [8, 12],           # Giảm độ sâu tối đa
    'min_samples_split': [5, 10],   # Tăng số mẫu tối thiểu để split
    'min_samples_leaf': [2, 4]      # Tăng số mẫu tối thiểu ở leaf
}

rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)  # n_jobs=-1 để tận dụng tất cả CPU
rf_grid = GridSearchCV(
    rf_model, 
    rf_params, 
    cv=3,                           # Giảm số fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,                      # Parallel processing
    verbose=1                       # Hiển thị tiến trình
)
rf_grid.fit(X_train_scaled, y_train)

print("\nTham số tốt nhất cho Random Forest:")
print(rf_grid.best_params_)

rf_pred = rf_grid.predict(X_test_scaled)
rf_metrics = evaluate_model(y_test, rf_pred, "Random Forest (optimized)")

# In ra feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_grid.best_estimator_.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTầm quan trọng của các features:")
print(feature_importance)


Huấn luyện Linear Regression...

Kết quả đánh giá mô hình Linear Regression:
MSE: 15.8344
RMSE: 3.9792
R2 Score: 0.7907
MAPE: 27.1056%

Huấn luyện Random Forest với GridSearchCV...
Fitting 3 folds for each of 16 candidates, totalling 48 fits

Tham số tốt nhất cho Random Forest:
{'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}

Kết quả đánh giá mô hình Random Forest (optimized):
MSE: 14.1109
RMSE: 3.7565
R2 Score: 0.8135
MAPE: 27.5690%

Tầm quan trọng của các features:
              feature  importance
0            distance    0.508445
1    distance_squared    0.472758
4                hour    0.010736
2  distance_passenger    0.004646
5         day_of_week    0.002042
6               month    0.001191
3     passenger_count    0.000181
